In [1]:
import math
import cv
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import cv2
import glob2

In [2]:
def find(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
    gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    bl = cv2.blur(gradient, (90, 90))
    (_, thresh) = cv2.threshold(bl, 70, 70, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 7))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations = 4)
    closed = cv2.dilate(closed, None, iterations = 4)
    return closed
    
def ans(closed):
    (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
    c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
    rect = cv2.minAreaRect(c)
    box = np.int0(cv2.boxPoints(rect))
    return box

    

In [3]:
anss = []
for name in glob2.glob("./Images/**.png"): 
    image = cv2.imdecode(np.fromfile(name, dtype=np.uint8), cv2.IMREAD_COLOR)
    box = ans(find(image))
    anss.append([name, "-" ,np.abs(box[0][0]), np.abs(box[0][1]), np.abs(box[1][0]), np.abs(box[1][1]), np.abs(box[2][0]),
                 np.abs(box[2][1]), np.abs(box[3][0]), np.abs(box[3][1]), "-"])


In [4]:
import pandas as pd
answer = pd.DataFrame(anss)

In [5]:
answer.to_csv('answer.csv', index = False, header = False)